In [1]:
import ee
import sys
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from eeSAR.s1 import s1_collection, s1_timescan
from datetime import datetime as dt, timedelta
import ee
ee.Initialize()

In [40]:
def addExtra(image):
    
    def addGEDI(image):
        
        gedi = ee.ImageCollection('users/potapovpeter/GEDI_V27').mosaic().rename('canopy_hei').unitScale(0,30);
        
        return image.addBands(gedi)
    
    
    def addHansen(image):
        

        coll_dict = {
            '2012': ee.Image("UMD/hansen/global_forest_change_2012_v1_0"),
            '2013': ee.Image("UMD/hansen/global_forest_change_2013_v1_1"),
            '2014': ee.Image("UMD/hansen/global_forest_change_2014_v1_2"),
            '2015': ee.Image("UMD/hansen/global_forest_change_2015_v1_3"),
            '2016': ee.Image("UMD/hansen/global_forest_change_2016_v1_4"),
            '2017': ee.Image("UMD/hansen/global_forest_change_2017_v1_5"),
            '2018': ee.Image("UMD/hansen/global_forest_change_2018_v1_6"),
            '2019': ee.Image("UMD/hansen/global_forest_change_2019_v1_7"),
        }
        
        #year = image.getInfo()['properties']['system:index'][17:21]
        #year = '2019' if year == '2020' else year
        
        hansen = coll_dict['2019']
        
        b3 = hansen.select(['last_b30'], ['B3'])
        b4 = hansen.select(['last_b40'], ['B4'])
        b5 = hansen.select(['last_b50'], ['B5'])
        b7 = hansen.select(['last_b70'], ['B7'])
        
        ndvi = (b4.subtract(b3)).divide(b4.add(b3)).rename('ndvi')
        ndmi = (b4.subtract(b5)).divide(b4.add(b5)).rename('ndmi')
        ndbri = (b4.subtract(b7)).divide(b4.add(b7)).rename('ndbri')
        
        return image.addBands(b3).addBands(b4).addBands(b5).addBands(b7).addBands(ndvi).addBands(ndmi).addBands(ndbri)
                              
    
    def addSRTM(image):

        srtm = ee.Image('USGS/SRTMGL1_003').resample()
        aspect = ee.Terrain.aspect(srtm).rename('aspect')
        slope = ee.Terrain.slope(srtm).rename('slope')

        return image.addBands(srtm.select('elevation').addBands(aspect).addBands(slope))
    
    
    def addChannelDistance(image):
        
        channels = ee.Image('users/marortpab/FAO/indonesia/cs4/stack_selected').select(4)
        return image.addBands(channels)
    
    image = addGEDI(image)
    image = addHansen(image)
    image = addSRTM(image)
    image = addChannelDistance(image)
    
    return image


def toLn(image):

    ln = image.select(['VV', 'VH']).log().rename(['VV', 'VH'])
    return image.addBands(ln, None, True)

def toLin(image):

    lin = ee.Image(10).pow(image.select(['VV', 'VH']).divide(10)).rename(['VV', 'VH'])
    return image.addBands(lin, None, True)

In [41]:
aoi = ee.FeatureCollection('users/marortpab/FAO/indonesia/cs4/selected_phu')
geom = ee.Geometry.Polygon([[[
    113.4710523892079,-3.1719616984149743],
    [114.9212477017079,-3.1719616984149743],
    [114.9212477017079,-1.8411739686394748],
    [113.4710523892079,-1.8411739686394748],
    [113.4710523892079,-3.1719616984149743]]]
)

# create time-series
tSeries = s1_collection.create(
        region=geom.bounds(),
        orbits=['ASCENDING', 'DESCENDING'],
        start_date='2014-01-01',
        end_date='2021-01-01',
        add_ratio=False,
        add_ND_ratio=False,
        speckle_filter='NONE',
        radiometric_correction='TERRAIN',
        slope_correction_dict={'model': 'surface', 'dem': 'USGS/SRTMGL1_003', 'buffer': 50},
        db=False,
        outlier_removal='AGGRESSIVE'
        ) #\
        #.filterMetadata('relativeOrbitNumber_start', 'equals', track_nr)
    
# create combined reducer
reducer = ee.Reducer.mean() \
    .combine(ee.Reducer.stdDev(), '', True) \
    .combine(ee.Reducer.percentile([5, 95]), '', True)

# create log timescan (k variables)
tScanLn = tSeries.map(toLn).select(['VV', 'VH']).reduce(reducer).select(
['VV_mean', 'VV_stdDev', 'VV_p5', 'VV_p95', 'VH_mean', 'VH_stdDev', 'VH_p5', 'VH_p95'],
['kVV_mean', 'kVV_stdDev', 'kVV_p5', 'kVV_p95', 'kVH_mean', 'kVH_stdDev', 'kVH_p5', 'kVH_p95'])

# create linear timescan
tScanLin = tSeries.map(toLin).select(['VV', 'VH']).reduce(reducer)

s1 = tScanLn.addBands(tScanLin)
stack = addExtra(s1)

In [42]:
stack.bandNames().getInfo()

['kVV_mean',
 'kVV_stdDev',
 'kVV_p5',
 'kVV_p95',
 'kVH_mean',
 'kVH_stdDev',
 'kVH_p5',
 'kVH_p95',
 'VV_mean',
 'VV_stdDev',
 'VV_p5',
 'VV_p95',
 'VH_mean',
 'VH_stdDev',
 'VH_p5',
 'VH_p95',
 'canopy_hei',
 'B3',
 'B4',
 'B5',
 'B7',
 'ndvi',
 'ndmi',
 'ndbri',
 'elevation',
 'aspect',
 'slope',
 'canal_d']

In [87]:
def add_class(feature):
    
    return feature.set('class', 0)

seed = 2020
training_data = ee.FeatureCollection('users/andreasvollrath/GWL_indonesia/mask_model')
print(training_data.size().getInfo())


training_data = training_data.map(add_class)
training_data = training_data.randomColumn('random', seed);

# Share of random training features, the % of features (being 1 = 100%) to include as training features, 1-n will be selected as testing features
n = 0.7

roi = stack.select(bands).sampleRegions(collection=training_data, scale=100, geometries=True, properties=['class'])
roi = roi.randomColumn('random', seed);
#//Select features to train the algorithm and left some out for testing
training = roi.filter(ee.Filter.lte('random',n)); 
testing  = roi.filter(ee.Filter.gt('random',n));



44


In [82]:
roi.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [109.09814546810559, 0.3067746695268166]},
 'id': '0000000000000000000b_0',
 'properties': {'B3': 25,
  'B4': 81,
  'B5': 79,
  'B7': 40,
  'aspect': 180,
  'canopy_hei': 0,
  'class': 0,
  'elevation': 5,
  'ndbri': 0.3388429880142212,
  'ndmi': 0.012500000186264515,
  'ndvi': 0.5283018946647644,
  'slope': 0.46373534202575684}}

In [88]:
bands = [#'kVV_mean',
 #'kVV_stdDev',
 #'kVV_p5',
 #'kVV_p95',
 #'kVH_mean',
 #'kVH_stdDev',
 #'kVH_p5',
 #'kVH_p95',
 #'VV_mean',
 #'VV_stdDev',
 #'VV_p5',
 #'VV_p95',
 #'VH_mean',
 #'VH_stdDev',
 #'VH_p5',
 #'VH_p95',
 'canopy_hei',
 'B3',
 'B4',
 'B5',
 'B7',
 'ndvi',
 'ndmi',
 'ndbri',
 'elevation',
 'aspect',
 'slope']

gamma = ee.Number(1).divide(ee.Number(training.size()))

classifier = ee.Classifier.svm(
    svmType='ONE_CLASS',
    kernelType='RBF',
    gamma=gamma,
    nu=0.05
).train(
    features=training,
    classProperty='class',
    inputProperties=bands
);

In [89]:
classified = stack.classify(classifier) #.add(ee.Image(1))

In [90]:
validated = testing.classify(classifier)
testAccuracy = validated.errorMatrix('class','classification')

In [91]:
print(testAccuracy.getInfo())
print(testAccuracy.accuracy().getInfo())

[[0, 16], [0, 0]]
0


In [93]:
task = ee.batch.Export.image.toAsset(
    assetId='users/andreasvollrath/mask_test3',
    image=classified,
    description='classified_asset',
    scale=100,
    region=geom,
    maxPixels=1e13
);
task.start()
task.status()

{'state': 'READY',
 'description': 'classified_asset',
 'creation_timestamp_ms': 1604596697279,
 'update_timestamp_ms': 1604596697279,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'T5QVSXZXJNR7WTMI4SMGKVL6',
 'name': 'projects/earthengine-legacy/operations/T5QVSXZXJNR7WTMI4SMGKVL6'}